<a href="https://colab.research.google.com/github/2024-RAG-COP/Stock-Chatbot/blob/jeongyeon/%08google_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install google-api-python-client

In [ ]:
from re import S
from googleapiclient.discovery import build
import html
import time
from datetime import datetime


def google_search(query, api_key, cse_id, start_date=None, end_date=None, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)

    # 날짜 범위가 지정된 경우 sort 매개변수 추가
    if start_date and end_date:
        kwargs['sort'] = f'date:r:{start_date}:{end_date}'

    res = service.cse().list(q=query, cx=cse_id, **kwargs).execute()
    return res  # 전체 응답을 반환

def get_news(query, start_date=None, end_date=None, total=None):
    try:
        time.sleep(1)  # API 요청 사이에 1초 지연 추가

        # 날짜 형식 변환 (YYYYMMDD)
        if start_date:
            start_date = datetime.strptime(start_date, "%Y-%m-%d").strftime("%Y%m%d")
        if end_date:
            end_date = datetime.strptime(end_date, "%Y-%m-%d").strftime("%Y%m%d")

        seen_titles = set()  # 중복 체크를 위한 집합
        items = []
        start_index = 1

        while len(items) < total:
            res = google_search(
                query,
                GOOGLE_API_KEY,
                GOOGLE_CSE_ID,
                num=10,  # 한 번에 10개씩 요청
                start=start_index,
                start_date=start_date,
                end_date=end_date
            )

            search_items = res.get('items', [])
            if not search_items:
                break  # 더 이상 결과가 없으면 종료

            for s in search_items:
                title = html.unescape(s.get('title', 'No title'))
                if title not in seen_titles:
                    seen_titles.add(title)  # 타이틀을 집합에 추가하여 중복 방지
                    link = s.get('link', 'No link')
                    snippet = html.unescape(s.get('snippet', 'No snippet'))
                    items.append(f"Title: {title}\nLink: {link}\nSnippet: {snippet}\n")

                    if len(items) == total:
                        break

            start_index += 10  # 다음 페이지로 이동
            time.sleep(1)  # API 요청 사이에 1초 지연 추가

        return "\n".join(items) if items else "No news found."
    except Exception as e:
        return f"Error fetching news: {str(e)}\nDetails: {e.args}"

def main():
    while True:
        param = input("종목명 입력: ")

        if param.lower() in ["exit", "quit"]:
            print("Goodbye")
            break

        if param.lower():
            symbol = param.strip()
            print(f"종목 검색: {symbol}")
            results = get_news(symbol, start_date="2024-08-01", end_date="2024-10-31", total=30)
            print("검색 결과:\n", results)
        else:
            continue

if __name__ == "__main__":
    main()

종목명 입력: microsoft
종목 검색: microsoft
검색 결과:
 Title: Stock market today: S&P 500 slumps as Microsoft, Meta lead tech ...
Link: https://www.investing.com/news/stock-market-news/stock-market-today-sp-500-slumps-as-microsoft-meta-lead-tech-wreck-3693435
Snippet: 2 days ago ... Microsoft, Meta slump on increased expenses outlook. Meta Platforms Inc (NASDAQ:META) slumped more than 4% after it racked up fewer than ...

Title: Microsoft print tops estimates but shares slide on AI capacity ...
Link: https://www.investing.com/news/stock-market-news/microsoft-q1-results-top-estimates-as-ai-services-grow-bolsters-performance-3692467
Snippet: 3 days ago ... Analysts polled by Investing.com anticipated EPS of $5.21 on revenue of $40.18 billion. Azure, Microsoft's cloud business grew 33%, just above ...

Title: Wedbush 'more bullish' on Microsoft, despite earnings noise By ...
Link: https://www.investing.com/news/stock-market-news/wedbush-more-bullish-on-microsoft-despite-earnings-noise-3694302
Snippet